In [13]:
from typing import TypedDict,List,Annotated
from langgraph.graph import END, START, StateGraph
import operator
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

In [14]:
class SimpleState(TypedDict):
    # count: int
    count: Annotated[int,operator.add]
    # summation: int
    summation: Annotated[int,operator.add]
    # tracking_sums: List['names']
    tracking_sums: Annotated[List[int],operator.concat]

In [86]:
def increment(state:SimpleState) -> SimpleState:
    # new_count=state['count'] + 1
    # new_sum=state['summation'] + new_count

    
    return {
        # "count": new_count,
        "count":1,
        # "summation": new_sum,
        "summation": state['count'] + 1,
        # "tracking_sums":state['tracking_sums'] +[new_sum],
        "tracking_sums": [state['count'] + 1]
    }

In [87]:
def should_continue(state:SimpleState):
    if state['count']<5:
        return "continue"
    else:
        return "stop"

In [88]:
graph=StateGraph(state_schema=SimpleState)

In [89]:
graph.add_node(node="increment",action=increment)

In [90]:
graph.set_entry_point(key="increment")

In [91]:
graph.add_conditional_edges(
                 source="increment",
                 path=should_continue,
                 path_map={
                     "continue": "increment",
                     "stop": END
                 }
             )

In [92]:
app=graph.compile()

In [93]:
# display(Image(data=app.get_graph(xray=True).draw_mermaid_png(max_retries=3)))

In [94]:
state={"count":0,"summation":0,"tracking_sums":[]}

In [95]:
result=app.invoke(input=state)

In [96]:
result

{'count': 5, 'summation': 15, 'tracking_sums': [1, 2, 3, 4, 5]}